In [2]:
import pyarrow.flight as flight
import pyarrow as pa
import duckdb
import pandas as pd

# Connect to the Flight server using the private IPv4 address
client = flight.connect("grpc://public_ip_address:8815")

# List available flights
print("Available Flights:")
flights = client.list_flights()
for f in flights:
    print(f.descriptor.path)

In [ ]:
# List available flights
print("Available Flights:")
flights = client.list_flights()
flight_descriptors = []
for f in flights:
    print(f.descriptor.path)
    flight_descriptors.append(f.descriptor)

if len(flight_descriptors) > 0:
    # Get the first flight from the list of available flights
    flight_descriptor = flight_descriptors[0]
    print("Selected Flight Descriptor:", flight_descriptor)

    # Get the flight info for the selected flight
    flight_info = client.get_flight_info(flight_descriptor)
    print("Flight Info:", flight_info)

    # Get the endpoint from the flight info
    endpoint = flight_info.endpoints[0]
    print("Endpoint:", endpoint)

    # Get the ticket from the endpoint
    ticket = endpoint.ticket
    print("Ticket:", ticket)
else:
    print("No available flights found.")

In [ ]:
if len(flight_descriptors) > 0:
    # Get the data from the flight
    print("Downloading data...")
    reader = client.do_get(ticket)

    # Read the data into an Arrow table
    table = reader.read_all()
    print("Data downloaded successfully.")
    print("Downloaded data:")
    print(table)
else:
    print("No flight selected.")

In [ ]:
# Create a DuckDB connection
con = duckdb.connect()

# Register the Arrow table as a view in DuckDB
con.register_arrow("my_table", table)

# Query the table using DuckDB and fetch the first 20 rows
query = "SELECT * FROM my_table LIMIT 20"
result = con.execute(query).fetchdf()

# Display the query result using Pandas
print("Query Result:")
print(pd.DataFrame(result))